# PatchSTG 分批加载训练 Notebook (方案 3)

**💡 混合模式：分天/分批加载数据，降低内存占用**

## 核心改进

- ✅ **不一次性加载全部数据**（7天 → 每次2天）
- ✅ **内存占用大幅降低**（50GB+ → ~10GB）
- ✅ **代码改动最小**（~50行）
- ✅ **保留所有现有逻辑**

## 配置信息

- 城市：北京 (adcode = 110000)
- 总日期：2025-09-19 ~ 2025-09-25 (7天)
- **每次加载：2天数据**
- 数据划分：60% / 20% / 20%

## 1. 环境准备和导入

In [ ]:
import os
import sys
import math
import time
import random
import configparser
from datetime import datetime, timedelta

import numpy as np
import torch
import torch.nn as nn
from tqdm.notebook import tqdm

# 添加项目路径
sys.path.insert(0, os.path.dirname(os.path.abspath('.')))

from models.model import PatchSTG
from lib.utils import log_string, _compute_loss, metric
from lib.odps_data_loader import ODPSDataLoader

print("=" * 80)
print("🚀 PatchSTG 分批加载训练 (方案 3)")
print("=" * 80)
print(f"开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 设备: {torch.cuda.get_device_name(0)}")
print()

## 2. 配置参数

In [ ]:
# 读取配置文件
config_file = 'config/ODPS.conf'
config = configparser.ConfigParser()
config.read(config_file)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
print(f"📅 训练时间戳: {timestamp}")

# 训练参数
args = {
    # 训练配置
    'cuda': config['train']['cuda'],
    'seed': int(config['train']['seed']),
    'batch_size': int(config['train']['batch_size']),
    'max_epoch': int(config['train']['max_epoch']),
    'learning_rate': float(config['train']['learning_rate']),
    'weight_decay': float(config['train']['weight_decay']),
    
    # ODPS 数据配置
    'odps_project': config['data']['odps_project'],
    'odps_endpoint': config['data']['odps_endpoint'],
    'odps_table': config['data']['odps_table'],
    'odps_meta_table': config['data']['odps_meta_table'],
    'adcode': config['data']['adcode'],
    'start_date': config['data']['start_date'],  # 总日期范围
    'end_date': config['data']['end_date'],
    'input_len': int(config['data']['input_len']),
    'output_len': int(config['data']['output_len']),
    'train_ratio': float(config['data']['train_ratio']),
    'val_ratio': float(config['data']['val_ratio']),
    'test_ratio': float(config['data']['test_ratio']),
    
    # 模型参数
    'layers': int(config['param']['layers']),
    'tem_patchsize': int(config['param']['tps']),
    'tem_patchnum': int(config['param']['tpn']),
    'factors': int(config['param']['factors']),
    'recur_times': int(config['param']['recur']),
    'spa_patchsize': int(config['param']['sps']),
    'spa_patchnum': int(config['param']['spn']),
    'tod': int(config['param']['tod']),
    'dow': int(config['param']['dow']),
    'input_dims': int(config['param']['id']),
    'node_dims': int(config['param']['nd']),
    'tod_dims': int(config['param']['td']),
    'dow_dims': int(config['param']['dd']),
    
    # 🆕 分批加载配置
    'days_per_batch': 2,  # 每次加载2天数据
    
    # 文件路径
    'model_file': f'./saved_models/odps_beijing_incremental_{timestamp}.pth',
    'log_file': f'./log/odps_beijing_incremental_{timestamp}',
}

print("\n📋 训练配置:")
print(f"  城市代码: {args['adcode']} (北京)")
print(f"  总日期范围: {args['start_date']} ~ {args['end_date']}")
print(f"  🆕 每批加载: {args['days_per_batch']} 天数据")
print(f"  Batch Size: {args['batch_size']}")
print(f"  Max Epochs: {args['max_epoch']}")
print()

os.makedirs('saved_models', exist_ok=True)
os.makedirs('log', exist_ok=True)

## 3. 生成日期批次

将总日期范围分割成多个小批次

In [ ]:
def generate_date_batches(start_date_str, end_date_str, days_per_batch):
    """
    生成日期批次
    
    参数:
        start_date_str: '20250919'
        end_date_str: '20250925'
        days_per_batch: 2
    
    返回:
        [('20250919', '20250920'), ('20250921', '20250922'), ...]
    """
    start_date = datetime.strptime(start_date_str, '%Y%m%d')
    end_date = datetime.strptime(end_date_str, '%Y%m%d')
    
    batches = []
    current_date = start_date
    
    while current_date <= end_date:
        batch_end = min(current_date + timedelta(days=days_per_batch - 1), end_date)
        batches.append((
            current_date.strftime('%Y%m%d'),
            batch_end.strftime('%Y%m%d')
        ))
        current_date = batch_end + timedelta(days=1)
    
    return batches

# 生成日期批次
date_batches = generate_date_batches(
    args['start_date'],
    args['end_date'],
    args['days_per_batch']
)

print(f"📅 将 {args['start_date']} ~ {args['end_date']} 分成 {len(date_batches)} 批:")
for i, (batch_start, batch_end) in enumerate(date_batches, 1):
    print(f"   批次 {i}: {batch_start} ~ {batch_end}")
print()

## 4. 设置随机种子和日志

In [ ]:
# 设置随机种子
if args['seed'] is not None:
    random.seed(args['seed'])
    np.random.seed(args['seed'])
    torch.manual_seed(args['seed'])
    if torch.cuda.is_available():
        torch.cuda.manual_seed(args['seed'])
        torch.backends.cudnn.deterministic = True
    print(f"✅ 随机种子已设置: {args['seed']}")

# 初始化日志
log = open(args['log_file'], 'w')
log_string(log, '=' * 80)
log_string(log, 'PatchSTG 分批加载训练 (方案 3)')
log_string(log, '=' * 80)
log_string(log, f"训练时间戳: {timestamp}")
log_string(log, f"每批加载: {args['days_per_batch']} 天")
log_string(log, f"总批次数: {len(date_batches)}")
log_string(log, '')

print(f"✅ 日志初始化: {args['log_file']}")
print()

## 5. 创建数据加载器（不立即加载数据）

In [ ]:
print("📊 创建数据加载器...")

# 创建数据加载器（不会立即加载数据）
data_loader = ODPSDataLoader(args, log)

# 🔑 关键：首次加载第一批数据，获取节点信息和归一化参数
print(f"\n🔄 首次加载第一批数据: {date_batches[0][0]} ~ {date_batches[0][1]}")
data_loader.load_data_for_date_range(date_batches[0][0], date_batches[0][1])

# 获取必要的全局信息（这些信息在整个训练中不变）
node_num = data_loader.node_num
mean, std = data_loader.get_normalization_params()
ori_parts_idx, reo_parts_idx, reo_all_idx = data_loader.get_patch_indices()

print(f"\n✅ 数据加载器初始化完成")
print(f"  节点数: {node_num}")
print(f"  归一化: mean={mean:.4f}, std={std:.4f}")
print(f"  首批样本: {data_loader.get_train_data()[0].shape[0]}")
print()

# 清空首批数据，准备训练
data_loader.clear_data()

## 6. 构建模型

In [ ]:
log_string(log, '\n------------ 构建模型 -------------')
print("🏗️  构建 PatchSTG 模型...\n")

device = torch.device(f"cuda:{args['cuda']}" if torch.cuda.is_available() else "cpu")
print(f"设备: {device}")

model = PatchSTG(
    args['output_len'], 
    args['tem_patchsize'], 
    args['tem_patchnum'],
    node_num,
    args['spa_patchsize'], 
    args['spa_patchnum'],
    args['tod'], 
    args['dow'],
    args['layers'], 
    args['factors'],
    args['input_dims'], 
    args['node_dims'], 
    args['tod_dims'], 
    args['dow_dims'],
    ori_parts_idx, 
    reo_parts_idx, 
    reo_all_idx
).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"模型参数: {total_params:,}")

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=args['learning_rate'],
    weight_decay=args['weight_decay']
)

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer,
    milestones=[1, 35, 40],
    gamma=0.5,
)

print("\n✅ 模型构建成功\n")

## 7. 定义验证函数

In [ ]:
def validate(model, valX, valY, valXTE, mean, std, device, batch_size, log):
    """
    验证函数
    """
    model.eval()
    num_val = valX.shape[0]
    pred = []
    label = []

    num_batch = math.ceil(num_val / batch_size)
    
    with torch.no_grad():
        for batch_idx in range(num_batch):
            start_idx = batch_idx * batch_size
            end_idx = min(num_val, (batch_idx + 1) * batch_size)

            X = valX[start_idx:end_idx]
            Y = valY[start_idx:end_idx]
            TE = torch.from_numpy(valXTE[start_idx:end_idx]).to(device)
            NormX = torch.from_numpy((X - mean) / std).float().to(device)

            y_hat = model(NormX, TE)
            pred.append(y_hat.cpu().numpy() * std + mean)
            label.append(Y)
    
    pred = np.concatenate(pred, axis=0)
    label = np.concatenate(label, axis=0)

    maes = []
    rmses = []
    mapes = []

    for i in range(pred.shape[1]):
        mae, rmse, mape = metric(pred[:, i, :], label[:, i, :])
        maes.append(mae)
        rmses.append(rmse)
        mapes.append(mape)
    
    mae, rmse, mape = metric(pred, label)
    maes.append(mae)
    rmses.append(rmse)
    mapes.append(mape)
    
    return np.stack(maes, 0), np.stack(rmses, 0), np.stack(mapes, 0)

print("✅ 验证函数定义完成")

## 8. 开始分批加载训练 🚀

**核心改进：每个 epoch 循环遍历所有日期批次**

In [ ]:
log_string(log, '\n' + '=' * 80)
log_string(log, '开始分批加载训练')
log_string(log, '=' * 80)

print("=" * 80)
print("🚀 开始分批加载训练")
print("=" * 80)
print(f"总 Epoch 数: {args['max_epoch']}")
print(f"每 Epoch 数据批次: {len(date_batches)}")
print(f"Batch Size: {args['batch_size']}")
print("=" * 80)
print()

min_val_loss = float('inf')
best_epoch = 0
train_losses = []
val_maes = []

try:
    for epoch in range(1, args['max_epoch'] + 1):
        epoch_start_time = time.time()
        model.train()
        
        epoch_train_loss = 0.0
        epoch_batch_count = 0
        
        # 🆕 每个 epoch 遍历所有日期批次
        print(f"\n{'='*80}")
        print(f"Epoch {epoch}/{args['max_epoch']}")
        print(f"{'='*80}\n")
        
        # 打乱日期批次顺序（增加随机性）
        shuffled_batches = date_batches.copy()
        random.shuffle(shuffled_batches)
        
        for batch_idx, (batch_start, batch_end) in enumerate(shuffled_batches, 1):
            print(f"\n📦 批次 {batch_idx}/{len(shuffled_batches)}: {batch_start} ~ {batch_end}")
            
            # 🔑 加载这批日期的数据
            data_loader.load_data_for_date_range(batch_start, batch_end)
            trainX, trainY, trainXTE, trainYTE = data_loader.get_train_data()
            
            print(f"   加载了 {trainX.shape[0]} 个训练样本")
            
            # 训练这批数据
            num_train = trainX.shape[0]
            num_batch = math.ceil(num_train / args['batch_size'])
            
            pbar = tqdm(
                range(num_batch),
                desc=f"   Epoch {epoch} - Batch {batch_idx}",
                leave=False
            )
            
            for sub_batch_idx in pbar:
                start_idx = sub_batch_idx * args['batch_size']
                end_idx = min(num_train, (sub_batch_idx + 1) * args['batch_size'])

                X = trainX[start_idx:end_idx]
                Y = trainY[start_idx:end_idx]
                TE = torch.from_numpy(trainXTE[start_idx:end_idx]).to(device)
                NormX = torch.from_numpy((X - mean) / std).float().to(device)
                Y_tensor = torch.from_numpy(Y).float().to(device)
                
                optimizer.zero_grad()
                y_hat = model(NormX, TE)
                loss = _compute_loss(Y_tensor, y_hat * std + mean)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
                optimizer.step()
                
                epoch_train_loss += loss.cpu().item()
                epoch_batch_count += 1
                
                pbar.set_postfix({'loss': f'{loss.item():.4f}'})
            
            # 🗑️ 释放当前批次的数据
            data_loader.clear_data()
            del trainX, trainY, trainXTE, trainYTE
            
            # 🧹 清理 GPU 缓存
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            print(f"   ✅ 批次 {batch_idx} 完成，内存已释放")
        
        # Epoch 结束，计算平均损失
        avg_train_loss = epoch_train_loss / epoch_batch_count
        train_losses.append(avg_train_loss)
        
        # 验证（使用最后一个日期批次的验证集）
        print(f"\n📊 Epoch {epoch} 训练完成，开始验证...")
        data_loader.load_data_for_date_range(date_batches[-1][0], date_batches[-1][1])
        valX, valY, valXTE, valYTE = data_loader.get_val_data()
        
        log_string(log, f'\nEpoch {epoch}/{args["max_epoch"]}:')
        log_string(log, f'  训练损失: {avg_train_loss:.4f}')
        log_string(log, f'  用时: {time.time() - epoch_start_time:.1f}s')
        
        maes, rmses, mapes = validate(
            model, valX, valY, valXTE, mean, std, device, args['batch_size'], log
        )
        
        val_mae = maes[-1]
        val_maes.append(val_mae)
        
        print(f"\n  训练损失: {avg_train_loss:.4f}")
        print(f"  验证 MAE: {val_mae:.4f}")
        print(f"  用时: {time.time() - epoch_start_time:.1f}s")
        
        # 清理验证数据
        data_loader.clear_data()
        
        # 更新学习率
        lr_scheduler.step()
        
        # 保存最佳模型
        if val_mae < min_val_loss:
            min_val_loss = val_mae
            best_epoch = epoch
            torch.save(model.state_dict(), args['model_file'])
            log_string(log, f'\n✅ 保存最佳模型 (Epoch {epoch}, MAE={val_mae:.4f})')
            print(f"✅ 保存最佳模型 (Epoch {epoch}, MAE={val_mae:.4f})")
    
    print("\n" + "=" * 80)
    print("🎉 训练完成！")
    print("=" * 80)
    print(f"最佳 Epoch: {best_epoch}")
    print(f"最佳验证 MAE: {min_val_loss:.4f}")
    print("=" * 80)
    
except KeyboardInterrupt:
    print("\n\n⚠️  训练被用户中断")
except Exception as e:
    print(f"\n❌ 训练出错: {e}")
    import traceback
    traceback.print_exc()

## 9. 测试集评估

In [ ]:
print("\n=" * 80)
print("📊 测试集评估")
print("=" * 80)

# 加载最佳模型
model.load_state_dict(torch.load(args['model_file'], map_location=device))
print(f"✅ 加载最佳模型: {args['model_file']}\n")

# 加载测试数据
data_loader.load_data_for_date_range(date_batches[-1][0], date_batches[-1][1])
testX, testY, testXTE, testYTE = data_loader.get_test_data()

model.eval()
num_test = testX.shape[0]
pred = []
label = []

num_batch = math.ceil(num_test / args['batch_size'])

print("预测中...")
with torch.no_grad():
    for batch_idx in tqdm(range(num_batch), desc="测试"):
        start_idx = batch_idx * args['batch_size']
        end_idx = min(num_test, (batch_idx + 1) * args['batch_size'])

        X = testX[start_idx:end_idx]
        Y = testY[start_idx:end_idx]
        TE = torch.from_numpy(testXTE[start_idx:end_idx]).to(device)
        NormX = torch.from_numpy((X - mean) / std).float().to(device)

        y_hat = model(NormX, TE)
        pred.append(y_hat.cpu().numpy() * std + mean)
        label.append(Y)

pred = np.concatenate(pred, axis=0)
label = np.concatenate(label, axis=0)

mae, rmse, mape = metric(pred, label)

print("\n" + "=" * 80)
print("📈 最终测试结果")
print("=" * 80)
print(f"  MAE:  {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAPE: {mape:.4f}")
print("=" * 80)
print()

## 10. 关闭日志

In [ ]:
log_string(log, f'\n结束时间: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
log.close()

print(f"\n✅ 日志已保存: {args['log_file']}")
print(f"✅ 模型已保存: {args['model_file']}")
print("\n🎉 全部完成！")